In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, make_scorer

In [2]:
class model_performance:
    
    def __init__(self, model):
        self.model = model

    def rmse(self, X, y):
        prediction = self.model.predict(X)
        mse = mean_squared_error(y, prediction)
        rmse = np.sqrt(mse)
        return rmse
    
    def r2(self, X, y):
        prediction = self.model.predict(X)
        r2 = np.sqrt(r2_score((y),(prediction)))
        return r2
    
    def cross_validation_scores(self, X, y):
        scores = cross_val_score(self.model, X, y, scoring="neg_mean_squared_error", cv=10)
        scores = np.sqrt(-scores)
        return scores
          
    def print_metrics(self, X, y):
        print(f'RMSE: {self.rmse(X, y)}')
        print(f'R2: {self.r2(X, y)}')
        print('\nCross_validation:')
        print(f'Mean: {self.cross_validation_scores(X, y).mean()}')
        print(f'SD: {self.cross_validation_scores(X, y).std()}')

### Import the dataset saved after EDA

In [3]:
diamonds = pd.read_csv('diamonds_eda.csv')
diamonds.head()

,carat,cut,color,clarity,price,lenght_x,width_y,depth_z
0,0.23,Ideal,E,SI2,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,335,4.34,4.35,2.75


### Split dataset to train and test set

Create X and y

In [4]:
X = diamonds.drop('price', axis=1)
y = diamonds['price']

In [5]:
X.shape, y.shape

((53917, 7), (53917,))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [7]:
X_train.head()

,carat,cut,color,clarity,lenght_x,width_y,depth_z
31745,0.35,Ideal,H,VVS1,4.55,4.58,2.78
20164,1.02,Very Good,F,VVS2,6.58,6.69,3.94
24520,1.52,Premium,D,SI1,7.55,7.51,4.50
42435,0.50,Ideal,E,SI1,5.09,5.13,3.15
10254,1.01,Premium,E,SI2,6.39,6.36,3.99


In [8]:
X_train.shape, y_train.shape

((43133, 7), (43133,))

### run Ordinal encoder on cut, color, clarity

In [9]:
categories = [['Fair', 'Good', 'Very Good', 'Ideal', 'Premium'],
             ['D', 'E', 'F', 'G', 'H', 'I', 'J'],
             ['I1','SI2', 'SI1', 'VS2',  'VS1', 'VVS2', 'VVS1', 'IF']]

In [10]:
obj_cols = ['cut', 'color', 'clarity']

In [11]:
ordinal_encoder = OrdinalEncoder(categories=categories)

In [12]:
X_train_cat = X_train[obj_cols]
X_test_cat = X_test[obj_cols]

In [13]:
ordinal_encoder.fit(X_train_cat)
X_train_oe = pd.DataFrame(ordinal_encoder.transform(X_train_cat))
X_test_oe = pd.DataFrame(ordinal_encoder.transform(X_test_cat))

In [14]:
X_train_oe.index = X_train.index
X_train_oe.columns = X_train_cat.columns

X_test_oe.index = X_test.index
X_test_oe.columns = X_test_cat.columns

### Normalise numerical variables with StandardScaler

In [15]:
num_X_train = X_train.drop(obj_cols, axis=1)
num_X_test = X_test.drop(obj_cols, axis=1)

In [16]:
scaler = StandardScaler()
num_X_train_scaled = pd.DataFrame(scaler.fit_transform(num_X_train))
num_X_train_scaled.columns = num_X_train.columns
num_X_train_scaled.index = num_X_train.index

num_X_test_scaled = pd.DataFrame(scaler.transform(num_X_test))
num_X_test_scaled.columns = num_X_test.columns
num_X_test_scaled.index = num_X_test.index

### Merge categorical and numerical variables

In [17]:
X_train = pd.concat([num_X_train_scaled, X_train_oe], axis=1)
X_test = pd.concat([num_X_test_scaled, X_test_oe], axis=1)

## Models

### Linear Regression

This will be a baseline model

In [19]:
lm = LinearRegression()
lm_model = lm.fit(X_train, y_train)

In [20]:
model_eval = model_performance(lm_model)
model_eval.print_metrics(X_train, y_train)

RMSE: 1223.4991085861132
R2: 0.9521052065649455

Cross_validation:
Mean: 1223.7471260610243
SD: 33.300100013323586


### Polynomial regression

In [21]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3)

In [22]:
X_train_poly = poly.fit_transform(X_train)
print(X_train_poly.shape)

(43133, 120)


In [23]:
pr_model = lm.fit(X_train_poly, y_train)

In [24]:
model_eval = model_performance(pr_model)
model_eval.print_metrics(X_train_poly, y_train)

RMSE: 604.780864300095
R2: 0.988511796480447

Cross_validation:
Mean: 689.2143960227021
SD: 112.00841564199301


### Batch gradient descent

In [23]:
from sklearn.linear_model import SGDRegressor
sgd_reg  = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.01)
sgd_reg.fit(X_train,y_train)

SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty=None, power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

In [24]:
model_eval = model_performance(sgd_reg)
model_eval.print_metrics(X_train, y_train)

RMSE: 1226.214678478257
R2: 0.9518869861496637

Cross_validation:
Mean: 1229.5729421658693
SD: 33.07360254689887


### Random Forest Regressor

In [18]:
rr  = RandomForestRegressor()
rr.fit(X_train,y_train)

RandomForestRegressor()

In [26]:
model_eval = model_performance(rr)
model_eval.print_metrics(X_train, y_train)

RMSE: 207.16143472353593
R2: 0.9986588932460532

Cross_validation:
Mean: 557.9777654624374
SD: 14.542014646646384


Random forest also allows to check feature importances.

In [30]:
pd.concat((pd.DataFrame(X_train.columns, columns = ['variable']), 
           pd.DataFrame(rr.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)

,variable,importance
0,carat,0.629945
2,width_y,0.259566
6,clarity,0.063108
5,color,0.032143
3,depth_z,0.007259
1,lenght_x,0.006016
4,cut,0.001963


In [21]:
X_train_rf = X_train[['carat', 'width_y', 'clarity', 'color']]
X_test_rf = X_test[['carat', 'width_y', 'clarity', 'color']]

In [22]:
rr_important_feats = RandomForestRegressor()
rr_important_feats.fit(X_train_rf,y_train)

RandomForestRegressor()

In [23]:
model_eval = model_performance(rr_important_feats)
model_eval.print_metrics(X_train_rf, y_train)

RMSE: 276.28733398402625
R2: 0.997613317016586

Cross_validation:
Mean: 610.9986647738992
SD: 16.837160674737973


### Let's try one of the ensemble machine learning algorithm - XGBoost

In [18]:
import xgboost as xgb

In [19]:
xgb_r = xgb.XGBRegressor(
    objective='reg:squarederror', 
    n_estimators=10,
    nthread=4
)

In [102]:
xgb_r.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [103]:
model_eval = model_performance(xgb_r)
model_eval.print_metrics(X_train, y_train)

RMSE: 585.0665775891654
R2: 0.9892525641384226

Cross_validation:
Mean: 613.4159140353904
SD: 13.925815931625383


In [22]:
parameters = {
    'max_depth': range(2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

grid_search = GridSearchCV(
    estimator=xgb_r, 
    param_grid=parameters, 
    #cv=5, 
    scoring='neg_mean_squared_error',
    return_train_score=True,
    n_jobs=-1,
    verbose=True)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


KeyboardInterrupt: 

### Random forest performs the best. I will try to further improve the model by testing different hyperparameters using GridSearchCV

In [19]:
param_grid = [
    {'n_estimators': [50, 100, 500, 1000], 'max_features': ['auto', 'sqrt', 'log2']}    
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, 
                          scoring='neg_mean_squared_error',
                          return_train_score=True,
                          n_jobs=-1)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(), n_jobs=-1,
             param_grid=[{'max_features': ['auto', 'sqrt', 'log2'],
                          'n_estimators': [50, 100, 500, 1000]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [20]:
grid_search.best_estimator_

RandomForestRegressor(n_estimators=500)

In [21]:
cvres = grid_search.cv_results_

In [22]:
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

562.5324906016921 {'max_features': 'auto', 'n_estimators': 50}
561.6659892759508 {'max_features': 'auto', 'n_estimators': 100}
559.9211184468177 {'max_features': 'auto', 'n_estimators': 500}
560.2582302533596 {'max_features': 'auto', 'n_estimators': 1000}
577.9584694896982 {'max_features': 'sqrt', 'n_estimators': 50}
572.0487866656403 {'max_features': 'sqrt', 'n_estimators': 100}
569.8017355801044 {'max_features': 'sqrt', 'n_estimators': 500}
569.112981911171 {'max_features': 'sqrt', 'n_estimators': 1000}
574.8929205646771 {'max_features': 'log2', 'n_estimators': 50}
571.3944064120652 {'max_features': 'log2', 'n_estimators': 100}
569.4279735924083 {'max_features': 'log2', 'n_estimators': 500}
568.4549817910143 {'max_features': 'log2', 'n_estimators': 1000}


In [23]:
final_model = grid_search.best_estimator_

In [25]:
prediction = final_model.predict(X_test)

In [26]:
model_eval = model_performance(final_model)
model_eval.print_metrics(X_test, y_test)

RMSE: 524.6361011146886
R2: 0.991048698928619

Cross_validation:
Mean: 576.5541407610643
SD: 31.753708379560734


In [28]:
import joblib

joblib.dump(final_model, "diamonds_final_model.pkl")

['diamonds_final_model.pkl']